# 모델링 전 사전세팅

In [2]:
# EDA 사전 세팅
%matplotlib inline

import time
import os
os.environ['KMP_WARNINGS'] = 'off'
from functools import reduce
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) ## 모든 열을 출력한다.
import seaborn as sns  #데이터 시각화 
sns.set(font="AppleGothic", 

        rc={"axes.unicode_minus":False},
        
        style='darkgrid')

# 모델링
import warnings
warnings.filterwarnings('ignore')
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from category_encoders.ordinal import OrdinalEncoder
from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.cluster import KMeans

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from skopt import BayesSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix, classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import mean_absolute_error


from xgboost import plot_importance

In [4]:
!pip install scikit-optimize

# 데이터 사전준비

In [3]:
# 오버샘플링 하지 않은 데이터셋
train = pd.read_csv("/Users/choejeehyuk/DNA/💳데이콘 신용카드 연체 예측 경진대회/Xy_train_nonsmote.csv")
train.head()
valid = pd.read_csv("/Users/choejeehyuk/DNA/💳데이콘 신용카드 연체 예측 경진대회/Xy_valid.csv")
train.head()
test = pd.read_csv("/Users/choejeehyuk/DNA/💳데이콘 신용카드 연체 예측 경진대회/X_test.csv")
valid.drop(['Unnamed: 0'], axis=1, inplace = True)
test.drop(['Unnamed: 0'], axis=1, inplace = True)
train.head()

train = train.astype({'reality' : int, 'income_type' : int, 'family_type' : int, 'house_type' : int, 'occyp_type' : int})
valid = valid.astype({'reality' : int, 'income_type' : int, 'family_type' : int, 'house_type' : int, 'occyp_type' : int})
test = test.astype({'reality' : int, 'income_type' : int, 'family_type' : int, 'house_type' : int, 'occyp_type' : int})

In [36]:
# 오버샘플링 하여 target 바율 맞춘 데이터 셋
train = pd.read_csv("/Users/choejeehyuk/DNA/💳데이콘 신용카드 연체 예측 경진대회/Xy_train_df.csv")
train.head()
valid = pd.read_csv("/Users/choejeehyuk/DNA/💳데이콘 신용카드 연체 예측 경진대회/Xy_valid_df.csv")
test = pd.read_csv("/Users/choejeehyuk/DNA/💳데이콘 신용카드 연체 예측 경진대회/X_test.csv")
test.drop(['Unnamed: 0'], axis=1, inplace = True)

train = train.astype({'car' : int, 'reality' : int, 'income_type' : int, 'family_type' : int, 'house_type' : int, 'occyp_type' : int})
valid = valid.astype({'car' : int, 'reality' : int, 'income_type' : int, 'family_type' : int, 'house_type' : int, 'occyp_type' : int})
test = test.astype({'reality' : int, 'income_type' : int, 'family_type' : int, 'house_type' : int, 'occyp_type' : int})

FileNotFoundError: [Errno 2] No such file or directory: '/Users/choejeehyuk/DNA/💳데이콘 신용카드 연체 예측 경진대회/Xy_train.csv'

In [44]:
#X_data = train.iloc[:, :-1]
#y_label = train.iloc[:, -1]
#X_train , X_valid , y_train , y_valid = train_test_split(X_data , y_label , test_size=0.2)

In [4]:
X_train = train.iloc[:, :-1]
X_valid = valid.iloc[:, :-1]
X_test = test.copy()
y_train = train.iloc[:, -1]
y_valid = valid.iloc[:, -1]

In [4]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21160 entries, 0 to 21159
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   reality        21160 non-null  int64  
 1   child_num      21160 non-null  float64
 2   income_type    21160 non-null  int64  
 3   family_type    21160 non-null  int64  
 4   house_type     21160 non-null  int64  
 5   occyp_type     21160 non-null  int64  
 6   month_ap       21160 non-null  float64
 7   family-child   21160 non-null  float64
 8   family_size    21160 non-null  float64
 9   income_total   21160 non-null  float64
 10  DAYS_EMPLOYED  21160 non-null  float64
 11  begin_month    21160 non-null  float64
 12  new_age        21160 non-null  float64
 13  aver_dep       21160 non-null  float64
 14  year_income    21160 non-null  float64
dtypes: float64(10), int64(5)
memory usage: 2.4 MB


In [6]:
X_test.info()
#y_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   reality        10000 non-null  int64  
 1   child_num      10000 non-null  float64
 2   income_type    10000 non-null  int64  
 3   family_type    10000 non-null  int64  
 4   house_type     10000 non-null  int64  
 5   occyp_type     10000 non-null  int64  
 6   month_ap       10000 non-null  float64
 7   family-child   10000 non-null  float64
 8   family_size    10000 non-null  float64
 9   income_total   10000 non-null  float64
 10  DAYS_EMPLOYED  10000 non-null  float64
 11  begin_month    10000 non-null  float64
 12  new_age        10000 non-null  float64
 13  aver_dep       10000 non-null  float64
 14  year_income    10000 non-null  float64
dtypes: float64(10), int64(5)
memory usage: 1.1 MB


In [12]:
xgb_pred_test = np.zeros((test.shape[0], 3))

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_valid)
y_pred_proba = model.predict_proba(X_valid)
confusion = confusion_matrix(y_valid, y_pred)
acc = accuracy_score(y_valid, y_pred)
logloss = log_loss(y_valid, y_pred_proba)

print(confusion, round(acc * 100, 2), logloss)

xgb_pred_test += model.predict_proba(test)

[14:08:07] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[  64   88  499]
 [  26  393  876]
 [  25   86 3234]] 69.76 0.7521843613723677


In [20]:
sub = pd.read_csv('/Users/choejeehyuk/분석에 필요한 데이터/신용커드 예측 경진대회/sample_submission.csv')

In [17]:
sub.iloc[:, 1:] = xgb_pred_test

In [18]:
sub

,index,0,1,2
0,26457,0.080742,0.148731,0.770527
1,26458,0.089658,0.155889,0.754453
2,26459,0.034229,0.081418,0.884353
3,26460,0.167461,0.157279,0.675260
4,26461,0.045887,0.096247,0.857866
...,...,...,...,...
9995,36452,0.115842,0.092561,0.791597
9996,36453,0.128363,0.271523,0.600114
9997,36454,0.036561,0.097620,0.865819
9998,36455,0.098931,0.272800,0.628269


In [15]:
sub.to_csv("xgb_no_smote_submission.csv", index=False)

In [31]:
def model_all(X_train, X_valid, y_train, y_valid):
    start_time = time.time()
    
    print('모델링 시작...............')
    print('----------------------------------------스태킹 모델----------------------------------------')
    # 개별 ML 모델을 위한 Classifier 생성.
    knn_clf  = KNeighborsClassifier(n_neighbors=4)
    rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
    dt_clf = DecisionTreeClassifier()
    ada_clf = AdaBoostClassifier(n_estimators=100)

    # 최종 Stacking 모델을 위한 Classifier생성. 
    lr_final = LogisticRegression(C=10)
    
    # 개별 모델들을 학습. 
    knn_clf.fit(X_train, y_train); knn_pred = knn_clf.predict(X_valid)
    rf_clf.fit(X_train , y_train); rf_pred = rf_clf.predict(X_valid)
    dt_clf.fit(X_train , y_train); dt_pred = dt_clf.predict(X_valid)
    ada_clf.fit(X_train, y_train); ada_pred = ada_clf.predict(X_valid)
    
    
    print('KNN 정확도: {0:.4f}'.format(accuracy_score(y_valid, knn_pred)))
    print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(y_valid, rf_pred)))
    print('결정 트리 정확도: {0:.4f}'.format(accuracy_score(y_valid, dt_pred)))
    print('에이다부스트 정확도: {0:.4f} :'.format(accuracy_score(y_valid, ada_pred)))
    
    pred = np.array([knn_pred, rf_pred, dt_pred, ada_pred])
    # transpose를 이용해 행과 열의 위치 교환. 컬럼 레벨로 각 알고리즘의 예측 결과를 피처로 만듦. 
    pred = np.transpose(pred)
    
    lr_final.fit(pred, y_valid)
    final = lr_final.predict(pred)

    print('스태킹 최종 메타 모델의 예측 정확도: {0:.4f}%'.format(round(accuracy_score(y_valid , final) *100, 2)))
    
    print('\n');
    print('----------------------------------Logistic Regression-----------------------------------')
    
    lr = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=0, solver='sag', tol=0.0001, verbose=0,
                   warm_start=False)
    
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_valid)
    y_pred_proba = lr.predict_proba(X_valid)
    confusion = confusion_matrix(y_valid, y_pred) 
    acc = accuracy_score(y_valid, y_pred)
    logloss = log_loss(y_valid, y_pred_proba)
    print(confusion, round(acc * 100, 2), logloss)
    print('\n');
    '''
    print('-----------------------------------SGD (SGDClassifier)----------------------------------')
    
    sgd = SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='cross_entropy',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=0, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)
    
    sgd.fit(X_train, y_train);prediction = sgd.predict(X_valid)
    y_pred_proba = sgd.predict_proba(X_valid)
    confusion = confusion_matrix(y_valid, y_pred) 
    acc = accuracy_score(y_valid, y_pred)
    logloss = log_loss(y_valid, y_pred_proba)
    print(confusion, round(acc * 100, 2), logloss)
    print('\n');
    '''
    print('-------------------------------KNN (KNeighborsClassifier)-------------------------------')
    
    knn = KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=9, p=2,
                     weights='uniform')
    
    knn.fit(X_train, y_train);prediction = knn.predict(X_valid)
    y_pred_proba = knn.predict_proba(X_valid)
    confusion = confusion_matrix(y_valid, y_pred) 
    acc = accuracy_score(y_valid, y_pred)
    logloss = log_loss(y_valid, y_pred_proba)
    print(confusion, round(acc * 100, 2), logloss)
    print('\n');
    print('-------------------------Decision Tree (DecisionTreeClassifier)-------------------------')
    
    dt = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0,
                       random_state=0, splitter='best')
    
    dt.fit(X_train, y_train);dt_pred = dt.predict(X_valid)
    
    y_pred_proba = dt.predict_proba(X_valid)
    confusion = confusion_matrix(y_valid, y_pred) 
    acc = accuracy_score(y_valid, y_pred)
    logloss = log_loss(y_valid, y_pred_proba)
    print(confusion, round(acc * 100, 2), logloss)
    
    print('\n');
    print('-----------------------------Cat Boost (Cat BoostClassifier)----------------------------')
    cat_model = CatBoostClassifier(iterations=100, logging_level='Silent')

    # 분류기를 훈련합니다.
    cat_model.fit(X_train, y_train)

    # 테스트 데이터에서 분류기를 사용하여 예측합니다.
    y_pred = cat_model.predict(X_valid)

    y_pred_proba = cat_model.predict_proba(X_valid)
    confusion = confusion_matrix(y_valid, y_pred) 
    acc = accuracy_score(y_valid, y_pred)
    logloss = log_loss(y_valid, y_pred_proba)
    print(confusion, round(acc * 100, 2), logloss)
    
    print('\n');
    print('---------------------------------XGBoost (XGBClassifier)--------------------------------')
    # create the model
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    y_pred_proba = model.predict_proba(X_valid)
    confusion = confusion_matrix(y_valid, y_pred) 
    acc = accuracy_score(y_valid, y_pred)
    logloss = log_loss(y_valid, y_pred_proba)
    print(confusion, round(acc * 100, 2), logloss)
    
    print('\n');
    print('--------------------------LGBM(Light Gradient Boosting Machine)-------------------------')
    model = LGBMClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    y_pred_proba = model.predict_proba(X_valid)
    confusion = confusion_matrix(y_valid, y_pred) 
    acc = accuracy_score(y_valid, y_pred)
    logloss = log_loss(y_valid, y_pred_proba)
    print(confusion, round(acc * 100, 2), logloss)
    
    end_time = time.time()
    print("Model execution time: {:.2f}s".format(end_time - start_time))

model_all(X_train, X_valid, y_train, y_valid)

모델링 시작...............
----------------------------------------스태킹 모델----------------------------------------
KNN 정확도: 0.4932
랜덤 포레스트 정확도: 0.6753
결정 트리 정확도: 0.5782
에이다부스트 정확도: 0.5375 :
스태킹 최종 메타 모델의 예측 정확도: 61.3800%


----------------------------------Logistic Regression-----------------------------------
[[  98  156  188]
 [ 177  393  319]
 [ 497  641 1187]] 45.9 1.0665032160563925


-------------------------------KNN (KNeighborsClassifier)-------------------------------
[[  98  156  188]
 [ 177  393  319]
 [ 497  641 1187]] 45.9 4.034891977592819


-------------------------Decision Tree (DecisionTreeClassifier)-------------------------
[[  98  156  188]
 [ 177  393  319]
 [ 497  641 1187]] 45.9 13.872872068267549


-----------------------------Cat Boost (Cat BoostClassifier)----------------------------
[[ 111   69  262]
 [  67  331  491]
 [ 141  217 1967]] 65.89 0.8335299351387384


---------------------------------XGBoost (XGBClassifier)--------------------------------
[16:03:29] WAR

In [11]:
X_train.head()

,car,reality,income_type,family_type,house_type,occyp_type,child_num,month_ap,family-child,family_size,income_total,DAYS_EMPLOYED,begin_month,new_age,aver_dep,year_income
0,0.0,1.0,0.0,0.0,1.0,8.0,0.648309,-0.730153,0.523754,0.764967,0.508421,-0.459964,-1.277577,-0.983449,-0.262049,0.155101
1,1.0,1.0,4.0,1.0,1.0,10.0,-0.661385,-0.730153,0.523754,-0.310591,2.476441,0.775603,-0.245901,1.210436,1.622181,-0.222538
2,0.0,1.0,0.0,1.0,1.0,14.0,-0.661385,1.026394,0.523754,-0.310591,0.071083,-0.224292,0.664401,0.061258,-0.014124,-0.176060
3,1.0,1.0,2.0,1.0,1.0,10.0,-0.661385,1.319152,0.523754,-0.310591,-0.366255,-0.218742,-0.003154,0.061258,-0.311634,-0.315496
4,0.0,1.0,4.0,1.0,1.0,6.0,1.958003,1.319152,0.523754,1.840524,0.727090,1.015544,-0.488648,-0.356625,-0.460389,-0.481750


In [8]:
cat_features = ['car', 'reality', 'income_type', 'family_type', 'house_type', 'occyp_type']

In [29]:
def modeling_boost(X_train, X_valid, y_train, y_valid):
    start_time = time.time()
    
    print('모델링 시작...............')
    print('-----------------------------Cat Boost (Cat BoostClassifier)----------------------------')
    cat_model = CatBoostClassifier(iterations=100, logging_level='Silent')
    
    # 분류기를 훈련합니다.
    cat_features = ['reality', 'income_type', 'family_type', 'house_type', 'occyp_type']
    cat_model.fit(X_train, y_train, cat_features=cat_features, verbose=50)

    # 테스트 데이터에서 분류기를 사용하여 예측합니다.
    y_pred = cat_model.predict(X_valid)

    y_pred_proba = cat_model.predict_proba(X_valid)
    confusion = confusion_matrix(y_valid, y_pred) 
    acc = accuracy_score(y_valid, y_pred)
    logloss = log_loss(y_valid, y_pred_proba)
    print(confusion, round(acc * 100, 2), logloss)
    
    print('---------------------------------XGBoost (XGBClassifier)--------------------------------')
    # create the model
    model = XGBClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    y_pred_proba = model.predict_proba(X_valid)
    confusion = confusion_matrix(y_valid, y_pred) 
    acc = accuracy_score(y_valid, y_pred)
    logloss = log_loss(y_valid, y_pred_proba)
    print(confusion, round(acc * 100, 2), logloss)
    
    print('\n');
    print('--------------------------LGBM(Light Gradient Boosting Machine)-------------------------')
    model = LGBMClassifier()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_valid)
    y_pred_proba = model.predict_proba(X_valid)
    confusion = confusion_matrix(y_valid, y_pred) 
    acc = accuracy_score(y_valid, y_pred)
    logloss = log_loss(y_valid, y_pred_proba)
    print(confusion, round(acc * 100, 2), logloss)
    
    
    
    end_time = time.time()
    print("Model execution time: {:.2f}s".format(end_time - start_time))

modeling_boost(X_train, X_valid, y_train, y_valid)

모델링 시작...............
-----------------------------Cat Boost (Cat BoostClassifier)----------------------------
[[  94   72  276]
 [  62  315  512]
 [ 158  189 1978]] 65.29 0.8425526847590287
---------------------------------XGBoost (XGBClassifier)--------------------------------
[16:02:29] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[[  83   62  297]
 [  33  314  542]
 [  70  142 2113]] 68.65 0.7781234815705536


--------------------------LGBM(Light Gradient Boosting Machine)-------------------------
[[  72   49  321]
 [  29  282  578]
 [  50   61 2214]] 70.24 0.784526506435204
Model execution time: 12.10s


In [35]:
clf = LGBMClassifier(objective='multiclass',
                         n_estimators=1000,
                         num_leaves=64,
                         learning_rate=0.1,
                         min_child_samples=10,
                         subsample=.5,
                         subsample_freq=1,
                         colsample_bytree=.8,
                         random_state=2023,
                         n_jobs=-1)
clf.fit(X_train, y_train,
        eval_set=[(X_valid, y_valid)],
        eval_metric='multiclass',
        early_stopping_rounds=10)
p_val = clf.predict(X_valid)
p_tst = clf.predict(X_test)

[1]	valid_0's multi_logloss: 1.07732
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 1.04956
[3]	valid_0's multi_logloss: 1.02912
[4]	valid_0's multi_logloss: 1.01616
[5]	valid_0's multi_logloss: 1.0051
[6]	valid_0's multi_logloss: 0.988338
[7]	valid_0's multi_logloss: 0.972134
[8]	valid_0's multi_logloss: 0.960109
[9]	valid_0's multi_logloss: 0.948366
[10]	valid_0's multi_logloss: 0.932498
[11]	valid_0's multi_logloss: 0.919457
[12]	valid_0's multi_logloss: 0.908737
[13]	valid_0's multi_logloss: 0.900321
[14]	valid_0's multi_logloss: 0.892834
[15]	valid_0's multi_logloss: 0.883917
[16]	valid_0's multi_logloss: 0.877396
[17]	valid_0's multi_logloss: 0.871122
[18]	valid_0's multi_logloss: 0.864454
[19]	valid_0's multi_logloss: 0.859997
[20]	valid_0's multi_logloss: 0.856065
[21]	valid_0's multi_logloss: 0.851701
[22]	valid_0's multi_logloss: 0.847344
[23]	valid_0's multi_logloss: 0.843615
[24]	valid_0's multi_logloss: 0.840323
[25]	valid_0's mul

ValueError: Number of features of the model must match the input. Model n_features_ is 16 and input n_features is 15 

In [6]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import log_loss



# Split the data into features and target
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

# Define the number of folds
n_folds = 5

# Create a StratifiedKFold object
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

# Initialize an empty list to store the scores
scores = {i:[] for i in set(y)}

# Iterate over the folds
for train_index, test_index in skf.split(X, y):
    # Split the data into training and test sets
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
    
    # Define your parameters
    params = {'boosting_type': 'gbdt', 'objective': 'multiclass', 'metric': {'multi_logloss'}, 'num_class':len(set(y))}
    gbm = lgb.train(params, lgb_train, num_boost_round=20, valid_sets=lgb_eval)

    # Make predictions on the test data
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    y_pred = pd.get_dummies(y_pred)
    
    for i in range(y_pred.shape[1]):
        score = log_loss(y_test, y_pred.iloc[:,i])
        scores[i].append(score)

# Print the mean and standard deviation of the scores for each class
for i in scores:
    print(f"log loss for class {i}: %0.2f (+/- %0.2f)" % (np.mean(scores[i]), np.std(scores[i]) * 2))

[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001976 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3839
[LightGBM] [Info] Number of data points in the train set: 22490, number of used features: 16
[LightGBM] [Warning] Find whitespaces in feature_names, replace with underlines
[LightGBM] [Info] Start training from score -1.098568
[LightGBM] [Info] Start training from score -1.098701
[LightGBM] [Info] Start training from score -1.098568
[1]	valid_0's multi_logloss: 1.07108
[2]	valid_0's multi_logloss: 1.04861
[3]	valid_0's multi_logloss: 1.02937
[4]	valid_0's multi_logloss: 1.01264
[5]	valid_0's multi_logloss: 0.997886
[6]	valid_0's multi_logloss: 0.984879
[7]	valid_0's multi_logloss: 0.9668
[8]	valid_0's multi_logloss: 0.950169
[9]	valid_0's multi_logl

ValueError: Data must be 1-dimensional

## Catboost 하이퍼 파라미터 튜닝 GridSearchCV

In [19]:
param_grid = {'iterations': [100, 200, 300],
              'depth': [3, 4, 5],
              'learning_rate': [0.1, 0.05, 0.01],
              'l2_leaf_reg': [3, 1, 5]}

In [20]:
# CatBoost 분류기를 생성합니다.
cat_model = CatBoostClassifier()

In [21]:
grid_search = GridSearchCV(cat_model, param_grid, cv=5, verbose = 0)
grid_search.fit(X_train, y_train, verbose=0)

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x7fab7321a730>,
             param_grid={'depth': [3, 4, 5], 'iterations': [100, 200, 300],
                         'l2_leaf_reg': [3, 1, 5],
                         'learning_rate': [0.1, 0.05, 0.01]})

In [22]:
print("Best parameters: {}".format(grid_search.best_params_))

Best parameters: {'depth': 5, 'iterations': 300, 'l2_leaf_reg': 1, 'learning_rate': 0.1}


In [23]:
best_model = grid_search.best_estimator_

In [24]:
best_model
y_pred = best_model.predict(X_valid)

acc = accuracy_score(y_valid, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 68.96%


## Catboost 하이퍼 파라미터 튜닝 BayesSearchCV

In [4]:
from catboost import CatBoostClassifier
from skopt import BayesSearchCV

In [26]:
# 하이퍼파라미터 검색 범위를 설정합니다.
param_dist = {'iterations': (100, 300, 500),
              'depth': (2, 5, 10),
              'learning_rate': (0.01, 0.05, 0.1),
              'l2_leaf_reg': (1, 3, 5)}

# CatBoost 분류기를 생성합니다.
cat_model = CatBoostClassifier()

In [27]:
# Bayesian Optimization을 사용하여 하이퍼파라미터를 튜닝합니다.
bayes_search = BayesSearchCV(cat_model, param_dist, n_iter=50, verbose=0)
bayes_search.fit(X_train, y_train, verbose=0)

/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
/Users/choejeehyuk/opt/anaconda3/lib/python3.9/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated

BayesSearchCV(estimator=<catboost.core.CatBoostClassifier object at 0x7fab69657a90>,
              search_spaces={'depth': (2, 5, 10), 'iterations': (100, 300, 500),
                             'l2_leaf_reg': (1, 3, 5),
                             'learning_rate': (0.01, 0.05, 0.1)})

In [28]:
# 가장 좋은 하이퍼파라미터를 출력합니다.
print("Best parameters: {}".format(bayes_search.best_params_))

Best parameters: OrderedDict([('depth', 10), ('iterations', 500), ('l2_leaf_reg', 1), ('learning_rate', 0.1)])


In [29]:
# 가장 좋은 성능을 내는 모델을 얻습니다.
best_model = bayes_search.best_estimator_

In [30]:
y_pred = best_model.predict(X_valid)

acc = accuracy_score(y_valid, y_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

Accuracy: 68.46%


## XGBClassifier RandomizedSearchCV

In [13]:
from sklearn.model_selection import RandomizedSearchCV

# define the parameter grid
param_grid = {
    'learning_rate': np.linspace(0.01, 0.3, num=30),
    'max_depth': [3, 4, 5, 6, 7, 8, 9],
    'n_estimators': [50, 100, 200, 300, 400, 500],
    'subsample': np.linspace(0.5, 0.9, num=5),
}

# create the model
model = XGBClassifier()

In [16]:
# create the randomized search cross-validator
rs_cv = RandomizedSearchCV(model, param_grid, cv=5, n_iter=50, verbose=0)

# fit the randomized search cross-validator on the data
rs_cv.fit(X_train, y_train, verbose=0)

[03:09:07] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:09:23] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:09:41] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1645117948562/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[03:09:58] WARNING: /Users/runner/miniforge3/c

RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           enable_categorical=False, gamma=None,
                                           gpu_id=None, importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate=None,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints...
                                           validate_parameters=None,
                                           verbosity=None),
                   n_iter=50,
                   param_distributions={'learning_rate': array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 ]),
                                        'max_depth': [3, 4, 5, 6, 7, 8, 9],
                                        'n_estimators': [50, 100, 200, 300, 400,
                                                         500],
                                        'subsample': array([0.5, 0.6, 0.7, 0.8, 0.9])})

In [17]:

# print the best hyperparameters
print(rs_cv.best_params_)

# make predictions on the test set
y_pred = rs_cv.predict(X_valid)

# evaluate predictions
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

{'subsample': 0.6, 'n_estimators': 400, 'max_depth': 9, 'learning_rate': 0.24999999999999997}
Accuracy: 68.41%


## LGBM(Light Gradient Boosting Machine) Hyperparameter optimization with GridSearchCV

In [10]:
# define the parameter grid
param_grid = {
    'learning_rate': [0.1, 0.05],
    'max_depth': [3, 4, 5],
    'n_estimators': [200, 300],
    'subsample': [0.8, 1.0],
}

# create the model
model = LGBMClassifier()

In [11]:
# create the grid search cross-validator
gs_cv = GridSearchCV(model, param_grid, cv=5, verbose=0)

# fit the grid search cross-validator on the data
gs_cv.fit(X_train, y_train, verbose=0)

GridSearchCV(cv=5, estimator=LGBMClassifier(),
             param_grid={'learning_rate': [0.1, 0.05], 'max_depth': [3, 4, 5],
                         'n_estimators': [200, 300], 'subsample': [0.8, 1.0]})

In [16]:
# print the best hyperparameters
print(gs_cv.best_params_)

# make predictions on the test set
y_pred = gs_cv.predict(X_valid)

# evaluate predictions
accuracy = accuracy_score(y_valid, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

{'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 300, 'subsample': 0.8}
Accuracy: 70.05%


In [9]:
import lightgbm as lgb

# define the search space
param_space = {
    'learning_rate': (0.01, 1.0, 0.05),
    'num_leaves': (2, 50, 100),
    'n_estimators': (50, 500),
    'min_child_samples': (5, 20),
    'max_depth': (1, 5, 10)
}


In [14]:
# define the model
lgb_model = lgb.LGBMClassifier()
start_time = time.time()
# perform the Bayesian optimization
opt = BayesSearchCV(lgb_model, param_space, n_iter=50, cv=5, verbose=0)
opt.fit(X_train, y_train, verbose=0)

end_time = time.time()

print("Model execution time: {:.2f}s".format((end_time - start_time)/60))

In [ ]:
# predict on the test set
y_pred = opt.predict(X_test)

# evaluate the model
acc = accuracy_score(y_test, y_pred)
print("Accuracy: ", acc)`